In [2]:
from io import StringIO
import pandas as pd

In [3]:
print(pd.get_option("display.max_columns"))
print(pd.get_option("display.max_rows"))
print(pd.set_option('display.max_columns', 1000))
print(pd.set_option('display.max_rows', 1000))

20
60
None
None


In [6]:
df= pd.read_excel('../data/dexp_watch_dummy.xls')
df.head()


,氏名コード,所属コード,開始日時,終了日時,取得件数,検索SQL
0,123,4000123,20230101,20230102,20,select hanbai1tbl.name from hanbai1tbl where h...
1,456,7000123,20230101,20230102,30,select nwi1tbl.name from nw1tbl
2,789,2000111,20230101,20230102,2,"select hanbai1tbl.name from hanbai1tbl,hanbai2..."
3,741,3000123,20230101,20230102,100,select nwi1tbl.name from nw1tbl


- re.IGNORECASE フラグを設定すると、パターンの中で記述したアルファベットの大文字と小文字を区別しなくなります。
−　https://www.megasoft.co.jp/mifes/seiki/meta.html
-  ソース　https://note.com/ai_memo/n/n2e959f0ae2bc

In [12]:
import re

for sql_text in df['検索SQL']:

    # 正規表現を使用して、SQL文からテーブル名を抽出する
    table_names = re.findall(r"\bFROM\s+([^\s;]+)", sql_text, flags=re.IGNORECASE) + \
                  re.findall(r"\bJOIN\s+([^\s;]+)", sql_text, flags=re.IGNORECASE)

    # ユニークなテーブル名のリストを表示する
    print(list(set(table_names)))

['hanbai1tbl']
['nw1tbl']
['hanbai1tbl,hanbai2tbl']
['nw1tbl']


In [24]:
#melt

df_v4_1tbl_melt = df_v4_1tbl.melt(id_vars=['業種別'],var_name="groupe")

df_v4_2tbl_melt = df_v4_2tbl.melt(id_vars=['業種別'],var_name="groupe")

df_v4_1tbl_melt.info(),df_v4_1tbl_melt.head()
#df_v4_2tbl_melt.info(),df_v4_2tbl_melt.head()

#df_v4_1tbl_melt.info(),df_v4_1tbl_melt.groupby("groupe").count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957 entries, 0 to 956
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   業種別     957 non-null    object
 1   groupe  957 non-null    object
 2   value   957 non-null    object
dtypes: object(3)
memory usage: 22.6+ KB


(None,
                 業種別 groupe value
 0                合計  集計企業数  2263
 1             製造業　計  集計企業数   551
 2  食料品、飲料・たばこ・飼料製造業  集計企業数    84
 3              繊維工業  集計企業数    14
 4     パルプ・紙・紙加工品製造業  集計企業数    15)

In [25]:
#可視化必要データへベース加工整形
#df_v4_1tbl_melt exclusive '合計'　'製造業　計'　'非製造業　計'
df_v4_1tbl_melt = df_v4_1tbl_melt[df_v4_1tbl_melt['業種別'] !=  '合計'] 
df_v4_1tbl_melt = df_v4_1tbl_melt[df_v4_1tbl_melt['業種別'] !=  '製造業　計'] 
df_v4_1tbl_melt = df_v4_1tbl_melt[df_v4_1tbl_melt['業種別'] !=  '非製造業　計'] 

#df_v4_2tbl_melt exclusive '合計'　'製造業　計'　'非製造業　計'
df_v4_2tbl_melt = df_v4_2tbl_melt[df_v4_2tbl_melt['業種別'] !=  '合計'] 
df_v4_2tbl_melt = df_v4_2tbl_melt[df_v4_2tbl_melt['業種別'] !=  '製造業　計'] 
df_v4_2tbl_melt = df_v4_2tbl_melt[df_v4_2tbl_melt['業種別'] !=  '非製造業　計'] 

## 可視化

In [26]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## IT関係諸経費内訳

## 産業における1企業当たりIt関係諸経費

In [28]:
#可視化必要データへ整形
df_v4_1tbl_melt_invst = df_v4_1tbl_melt[df_v4_1tbl_melt['groupe']=='IT関係諸経費合計（百万円）']

fig = px.bar(df_v4_1tbl_melt_invst, x='業種別',y ='value',color='業種別')

#軸の設定等 何故かエラー？？？
# fig.update_layout(width=900,height=500,margin=dict(l=50,r=50,b=40,t=60),
#                   yaxis=dict(range=[0,200]),xaxis=dict(dtick=1, autorange=True))

fig.update_yaxes(title_text="It関係諸経費（百万円）")

#データを並び替え
fig.update_layout(xaxis={'categoryorder':'total descending'})

#グラフの表示
fig.show()


## 産業における1企業当たり総従業者数

In [30]:
#可視化必要データへ整形
df_v4_1tbl_melt_employee = df_v4_1tbl_melt[df_v4_1tbl_melt['groupe']=='総従業者数（人）']

fig = px.bar(df_v4_1tbl_melt_employee, x='業種別',y ='value',color='業種別')
## 産業におけるIt関係諸経費
#軸の設定等 何故かエラー？？？
# fig.update_layout(width=900,height=500,margin=dict(l=50,r=50,b=40,t=60),
#                   yaxis=dict(range=[0,200]),xaxis=dict(dtick=1, autorange=True))

fig.update_yaxes(title_text="総従業者数（人）")

#データを並び替え
fig.update_layout(xaxis={'categoryorder':'total descending'})

#グラフの表示
fig.show()

## 産業における従業者一人当たりIT関係諸経費

In [29]:
#可視化必要データへ整形
df_v4_1tbl_melt_invst_perone = df_v4_1tbl_melt[df_v4_1tbl_melt['groupe']=='従業者一人当たりIT関係諸経費（千円）']

fig = px.bar(df_v4_1tbl_melt_invst_perone, x='業種別',y ='value',color='業種別')
## 産業におけるIt関係諸経費
#軸の設定等 何故かエラー？？？
# fig.update_layout(width=900,height=500,margin=dict(l=50,r=50,b=40,t=60),
#                   yaxis=dict(range=[0,200]),xaxis=dict(dtick=1, autorange=True))

fig.update_yaxes(title_text="従業者一人当たりIT関係諸経費（千円）")

#データを並び替え
fig.update_layout(xaxis={'categoryorder':'total descending'})

#グラフの表示
fig.show()

In [31]:

uriage_fig= make_subplots(rows=2, cols=2,
                          #column_widths=[0.7, 0.3],
                          subplot_titles=("1. It関係諸経費", "2. 総従業員数", "3. 一人当たりIT関係諸経費", "4. "),
                          #specs=[[{"secondary_y":True},{"secondary_y": True}],[{"secondary_y":True},{"secondary_y": True}]]
                         ) 

uriage_fig.update_layout(title_text="table1 総従業員数とIT関係諸経費", height=1100,width=1000)


#可視化必要データへ整形
df_v4_1tbl_melt_invst = df_v4_1tbl_melt[df_v4_1tbl_melt['groupe']=='IT関係諸経費合計（百万円）']
#layout_updateで1つ目のみしかソートできないので、データをソートしてしまう。
df_v4_1tbl_melt_invst = df_v4_1tbl_melt_invst.sort_values('value',ascending=False)

#可視化必要データへ整形
df_v4_1tbl_melt_employee = df_v4_1tbl_melt[df_v4_1tbl_melt['groupe']=='総従業者数（人）']
#layout_updateで1つ目のみしかソートできないので、データをソートしてしまう。
df_v4_1tbl_melt_employee = df_v4_1tbl_melt_employee.sort_values('value',ascending=False)

#可視化必要データへ整形
df_v4_1tbl_melt_invst_perone = df_v4_1tbl_melt[df_v4_1tbl_melt['groupe']=='従業者一人当たりIT関係諸経費（千円）']
#layout_updateで1つ目のみしかソートできないので、データをソートしてしまう。
df_v4_1tbl_melt_invst_perone = df_v4_1tbl_melt_invst_perone.sort_values('value',ascending=False)





# uriage_fig.add_trace(go.Bar(x = df_v4_1tbl_melt['業種別'],y=df_v4_1tbl_melt['value']),row=1, col=1)
# uriage_fig.add_trace(go.Scatter(x = df_uriage.index,y=df_uriage['総資産'],name="総資産",),row=1, col=1,secondary_y=True)
# uriage_fig.add_trace(go.Scatter(x = df_uriage.index,y=df_uriage['純資産'],name="純資産"),row=1, col=1,secondary_y=True)

#subplots 1-1
uriage_fig.add_trace(go.Bar(x = df_v4_1tbl_melt_invst['業種別'],y=df_v4_1tbl_melt_invst['value'],),row=1, col=1)

#subplots 1-2
uriage_fig.add_trace(go.Bar(x = df_v4_1tbl_melt_employee['業種別'],y=df_v4_1tbl_melt_employee['value']),row=1, col=2)

#subplots 2-1
uriage_fig.add_trace(go.Bar(x = df_v4_1tbl_melt_invst_perone['業種別'],y=df_v4_1tbl_melt_invst_perone['value']),row=2, col=1)


#subplots 2-2



#データを並び替え 1つ目のグラフのみしかソートできないので、使わない。。。
#uriage_fig.update_layout(xaxis={'categoryorder':'total descending'})

#layout
uriage_fig.update_yaxes(title="百万円",row=1, col=1)
uriage_fig.update_yaxes(title="人数", row=1, col=2)
uriage_fig.update_yaxes(title="千円",row=2, col=1)

#IT投資が大きい業種は、プラスチック製造業、映像メディア放送、輸送用機械、金融保険、エネルギーインフラ、鉄、化学あたり
#一人あたりのIT投資とすると、輸送用機械・鉄・化学は落ちる。
#仮説 輸送用機械・鉄・化学は人が多い。やはり人作業が多くIT化も進んでいなさそう。今後は、可能性はIT化を進める可能性はあるってことか？

## IT関係諸経費内訳

In [39]:
#可視化必要データへ整形
#必要カラム抽出 産業における従業者一人当たりIT関係諸経費ムのみ抽出
df_v4_1tbl_melt_v2 = df_v4_1tbl_melt[(df_v4_1tbl_melt['groupe'] == 'コンピュータ・周辺機器関連計（百万円）') | (
        df_v4_1tbl_melt['groupe'] ==  '通信機器関連費用計（百万円）')  |(
        df_v4_1tbl_melt['groupe'] ==  'その他の情報機器関連費用計（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  'ソフトウェア関連費用計（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  '処理サービス料（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  '運用保守委託料（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  '情報セキュリティ関連サービス料（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  'その他サービス関連支出（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  'その他費用（百万円）')
     ]

fig = px.bar(df_v4_1tbl_melt_v2, x='業種別',y ='value',color='groupe')

fig.update_yaxes(title_text="IT関係諸経費内訳")

#データを並び替え
fig.update_layout(xaxis={'categoryorder':'total descending'})

#グラフの表示
fig.show()

fig2 = px.bar(df_v4_1tbl_melt_v2, x='groupe',y ='value',color='業種別')


fig2.update_yaxes(title_text="IT関係諸経費内訳")

#データを並び替え
fig2.update_layout(xaxis={'categoryorder':'total descending'})

#グラフの表示
fig2.show()

#ソフトウェア費用など減価償却かそれ以外かを確認する。
#必要カラ## 産業における従業者一人当たりIT関係諸経費のみ抽出
df_v4_1tbl_melt_v3 = df_v4_1tbl_melt[(df_v4_1tbl_melt['groupe'] == 'コンピュータ・周辺機器関連費（％）') | (
        df_v4_1tbl_melt['groupe'] ==  '通信機器関連費用（％）')  |(
        df_v4_1tbl_melt['groupe'] ==  'その他の情報機器関連費用（％）')|(
        df_v4_1tbl_melt['groupe'] ==  'ソフトウェア関連費用（％）')|(
        df_v4_1tbl_melt['groupe'] ==  '処理サービス料（％）')|(
        df_v4_1tbl_melt['groupe'] ==  '運用保守委託料（％）')|(
        df_v4_1tbl_melt['groupe'] ==  '情報セキュリティ関連サービス料（％）')|(
        df_v4_1tbl_melt['groupe'] ==  'その他サービス関連支出（％）')|(
        df_v4_1tbl_melt['groupe'] ==  'その他費用（％）')
     ]

fig3 = px.bar(df_v4_1tbl_melt_v3, x='業種別',y ='value',color='groupe')

fig3.update_yaxes(title_text="IT関係諸経費内訳")

#データを並び替え
fig3.update_layout(xaxis={'categoryorder':'total descending'})

#グラフの表示
fig3.show()



#必要カラム抽出　産業における従業者一人当たりIT関係諸経費ムのみ抽出
df_v4_1tbl_melt_v4 = df_v4_1tbl_melt[(df_v4_1tbl_melt['groupe'] == 'コンピュータ・周辺機器関連費用当期減価償却費（百万円）') | (
        df_v4_1tbl_melt['groupe'] ==  'コンピュータ・周辺機器関連費用その他（百万円）')  |(
        df_v4_1tbl_melt['groupe'] ==  '通信機器関連費用計当期減価償却費（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  '通信機器関連費用計その他（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  'その他の情報機器関連費用当期減価償却費（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  'その他の情報機器関連費用その他（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  'ソフトウェア関連費用当期減価償却費（百万円）')|(
        df_v4_1tbl_melt['groupe'] ==  'ソフトウェア関連費用その他（百万円）')
     ]

fig4 = px.bar(df_v4_1tbl_melt_v4, x='業種別',y ='value',color='groupe')

fig4.update_yaxes(title_text="IT関係諸経費内訳")

#データを並び替え
fig4.update_layout(xaxis={'categoryorder':'total descending'})

#グラフの表示
fig4.show()



#１、２、３コンピュータや通信機器などハードでよりフトウェア関連費用と運用保守委託料の占める割合が高い
#　　　石油石炭製造業でソフトウェアがダントツ高いのは何故？３でみるとほぼソフトウェアの減価償却なので、過去の大型ソフトウェアの償却で苦しめられてる？
#４．電気・輸送・鉄鋼は、減価償却以外のソフトウェア関連費用が大きい

## 産業における1企業当たりIT関係諸経費(table2)

In [32]:
#2table　
#table1のIT関係諸経費とさほど大差はないので、table1もtable2も信憑性はある。

#可視化必要データへ整形
df_v4_2tbl_melt_sales = df_v4_2tbl_melt[df_v4_2tbl_melt['groupe']=='年間事業収入（百万円）']
df_v4_2tbl_melt_invst2 = df_v4_2tbl_melt[df_v4_2tbl_melt['groupe']=='IT関係諸経費(百万円)']
df_v4_2tbl_melt_invst_rate_persales = df_v4_2tbl_melt[df_v4_2tbl_melt['groupe']=='対年間事業収入比（％）']


fig = px.bar(df_v4_2tbl_melt_invst2, x=df_v4_2tbl_melt_invst2['業種別'],y =df_v4_2tbl_melt_invst2['value'],color=df_v4_2tbl_melt_invst2['業種別'])

#軸の設定等 何故かエラー？？？
# fig.update_layout(width=900,height=500,margin=dict(l=50,r=50,b=40,t=60),
#                   yaxis=dict(range=[0,200]),xaxis=dict(dtick=1, autorange=True))

fig.update_yaxes(title_text="百万円")

#データを並び替え
fig.update_layout(xaxis={'categoryorder':'total descending'})

#グラフの表示
fig.show()



In [34]:

uriage_fig= make_subplots(rows=2, cols=2,
                          #column_widths=[0.7, 0.3],
                          subplot_titles=("1. IT関係諸経費", "2. 年間事業収入" ,"3. 対年間事業収入比", "4. "),
                          #specs=[[{"secondary_y":True},{"secondary_y": True}],[{"secondary_y":True},{"secondary_y": True}]]
                         ) 

uriage_fig.update_layout(title_text="table2 年間収入とIT関連書経費の比率", height=1100,width=1000)


#可視化必要データへ整形
df_v4_2tbl_melt_invst2 = df_v4_2tbl_melt[df_v4_2tbl_melt['groupe']=='IT関係諸経費(百万円)']
df_v4_2tbl_melt_invst2 = df_v4_2tbl_melt_invst2.sort_values('value',ascending=False)

df_v4_2tbl_melt_sales = df_v4_2tbl_melt[df_v4_2tbl_melt['groupe']=='年間事業収入（百万円）']
df_v4_2tbl_melt_sales = df_v4_2tbl_melt_sales.sort_values('value',ascending=False)

df_v4_2tbl_melt_invst_rate_persales = df_v4_2tbl_melt[df_v4_2tbl_melt['groupe']=='対年間事業収入比（％）']
df_v4_2tbl_melt_invst_rate_persales = df_v4_2tbl_melt_invst_rate_persales.sort_values('value',ascending=False)

#subplots 1-1
uriage_fig.add_trace(go.Bar(x = df_v4_2tbl_melt_invst2['業種別'],y=df_v4_2tbl_melt_invst2['value']),row=1, col=1)

#subplots 1-2
uriage_fig.add_trace(go.Bar(x = df_v4_2tbl_melt_sales['業種別'],y=df_v4_2tbl_melt_sales['value'],),row=1, col=2)

#subplots 2-1
uriage_fig.add_trace(go.Bar(x = df_v4_2tbl_melt_invst_rate_persales['業種別'],y=df_v4_2tbl_melt_invst_rate_persales['value']),row=2, col=1)

#subplots 2-2



#データを並び替え 1つ目のグラフのみしかソートできないので、使わない。。。
#uriage_fig.update_layout(xaxis={'categoryorder':'total descending'})

#layout
uriage_fig.update_yaxes(title="百万円",row=1, col=1)
uriage_fig.update_yaxes(title="百万円",row=1, col=2)
uriage_fig.update_yaxes(title="％", row=2, col=1)

##　石油石炭製品製造業は売上の中でのIT投資比率がダントツ高い。これは何故？？？
#その他の医療、情報サービス、金融業、メディア・放送はわかる。